# Task 2
I will include some of the steps in the task 2 to start from the proper dataset

In [ ]:
#import statements
import pandas as pd
import tensorflow as tf
import re
import spacy
import nltk
from nltk.corpus import stopwords
import numpy as np

In [ ]:
def word_count_col(df, text, new_col_name):
    """This function will define the number of words in each row of the considered dataframe
    df: dataframe to use
    text: the text column in the dataframe
    new_col_name: the name to which associate the column of word count"""
    tot = len(df)
    words = []
    for i in range(tot):
        words.append(len(df[text][i].split()))
    df[new_col_name] = words
    return df

def preprocess(input_text):
    """This function will allow us to perform all the preprocessing steps to reduce noise in text data. This is fundamental to perform text analysis.
    The function will return a list of cleaned sentences"""

    # Normalize to lowercase
    txt = input_text.lower()

    # Remove URLs
    txt = re.sub('https?://\S+|www\.\S+', '', txt)

    # Remove numbers
    txt = re.sub(r"\d", "", txt)

    # Remove punctuation
    txt = re.sub('\n', '', txt)
    txt = re.sub(r"[^A-Za-z]+", " ", txt)

    # Remove stopwords
    stop_words = [w.lower() for w in stopwords.words('english')]
    stop_words = set(stop_words).difference(set(['against','above','below', 'over', 'under']))
    txt = ' '.join([w for w in txt.split() if w not in stop_words and w !='user'])

    # Lemmatization
    lemmas = [token.lemma_ for token in nlp(txt)]
    txt = " ".join(lemmas)

    return txt

In [ ]:
!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

In [ ]:
data = word_count_col(data,'Text','pre_word_count')

In [ ]:
documents = data["Text"].tolist()

In [ ]:
nltk.download('stopwords')
%time clean_des = [preprocess(string) for string in documents[:]]

In [ ]:
data['clean_text'] = clean_des
data = word_count_col(data,'clean_text','post_word_count')

In [ ]:
data


# Task 3


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, RNN, SimpleRNN
from sklearn.model_selection import train_test_split 


In [ ]:
clean_text = data.clean_text.to_list()

In [ ]:
#this cell is not useful for tokenizing and padding, but just to evaluate how many tokens we have before to proceed
#takes time to run; the result is 49483

#tokens = [[token.text for token in nlp(sentence)] for sentence in clean_text]
#count = 0
#for l in tokens:
#  for w in l:
#    count += 1 
#print(count)

In [ ]:
# TOKENIZING: First trial, get a strange result as i am not able to include more words reducing the num_words which should represent the minimum frequency for a word to be included 
# tokenizer = Tokenizer(num_words=100000000, lower= 1, oov_token="<OOV>")
# tokenizer.fit_on_texts(clean_text)
# word_index = tokenizer.word_index
# print(len(word_index))

In [ ]:
# TOKENIZING 
tk = Tokenizer()
tk.fit_on_texts(clean_text)

tk.num_words = 1000
word_index = tk.word_index
#print(word_index)

In [ ]:
# SEQUENCES 
sequences = tk.texts_to_sequences(clean_text)
print(clean_text)
print(word_index)
print(sequences)

In [ ]:
# PADDING 
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding="post")
print(padded_inputs)
X = padded_inputs

### LSTM 

In [ ]:
model_LSTM = Sequential()
model_LSTM.add(Embedding(500, 120, input_length = X.shape[1]))
model_LSTM.add(SpatialDropout1D(0.4))
model_LSTM.add(LSTM(176, dropout=0.2, recurrent_dropout=0.2))
model_LSTM.add(Dense(2,activation='softmax'))
model_LSTM.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model_LSTM.summary())

In [ ]:
y = pd.get_dummies(data['Sentiment'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, random_state = 42)

In [ ]:
batch_size=32
# model_LSTM.fit(X_train, y_train, epochs = 10, batch_size=batch_size, verbose = 'auto')
model_LSTM.fit(X_train, y_train, epochs = 1, batch_size=batch_size, verbose = 'auto')

In [ ]:
loss, accuracy = model_LSTM.evaluate(X_test,y_test)
print(accuracy)

In [ ]:
# Confusion Matrix 
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
y_test = y_test.iloc[:,1]
y_pred = model_LSTM.predict(X_test)
confusion_matrix(y_test, y_pred)

### Bi-LSTM 

In [ ]:
from keras.layers import Bidirectional

In [ ]:
model_BI = Sequential()
model_BI.add(Embedding(500, 120, input_length = X.shape[1]))
model_BI.add(SpatialDropout1D(0.4))
model_BI.add(Bidirectional(LSTM(176, dropout=0.2, recurrent_dropout=0.2)))
model_BI.add(Dense(2,activation='softmax'))
model_BI.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model_BI.summary())

In [ ]:
batch_size=32
# model_BI.fit(X_train, y_train, epochs = 10, batch_size=batch_size, verbose = 'auto')
model_BI.fit(X_train, y_train, epochs = 1, batch_size=batch_size, verbose = 'auto')

In [ ]:
loss, accuracy = model_BI.evaluate(X_test,y_test)
print(accuracy)

In [ ]:
y_pred = model_BI.predict(X_test)

In [ ]:
y_test = y_test.iloc[:,1]
y_test=np.argmax(y_test, axis=1)
confusion_matrix(y_test, y_pred)



---



In [ ]:
model_SimpleRNN = Sequential()
model_SimpleRNN.add(Embedding(500, 120, input_length = X.shape[1]))
model_SimpleRNN.add(SpatialDropout1D(0.4))
model_SimpleRNN.add(SimpleRNN(176))
model_SimpleRNN.add(Dense(2,activation='softmax'))
model_SimpleRNN.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model_SimpleRNN.summary())

In [ ]:
batch_size=32
model_SimpleRNN.fit(X_train, y_train, epochs = 1, batch_size=batch_size, verbose = 'auto')

In [ ]:
model_SimpleRNN.evaluate(X_test,y_test)

In [ ]:
y_pred = model_SimpleRNN.predict(X_test)

In [ ]:
y_pred=np.argmax(y_pred, axis=1)


In [ ]:
loss, accuracy = model_SimpleRNN.evaluate(X_test,y_test)
print(accuracy)

In [ ]:
y_test = y_test.iloc[:,1]
y_test=np.argmax(y_test, axis=1)
confusion_matrix(y_test, y_pred)